Given any type of comments, This program can detect the profanity of the comment along with different metrics.A good and long list containing profanity words is required.

In [79]:
import pandas as pd
df=pd.read_csv("comments.csv")

In [80]:
import re
#cleaning comment from links,tags and numbers
def clean_text(comment):
    return ''.join(re.sub(r"(@[A-Za-z0-9]+)|(http\S+)|(\$[A-Za-z0-9]+)|([0-9]+)","",comment))
def remove_special_chars(comment):# unrolls hastags and also removes symbols
    for remove in map(lambda r: re.compile(re.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#"]):
        comment.replace(remove, "", inplace=True)
    return comment

In [81]:
df["clean_text"]=df['Comment'].apply(lambda row: clean_text(row))
remove_special_chars(df.clean_text)
df.head()

,User,Comment,clean_text
0,1,Hello bitch,Hello bitch
1,2,bye bye love,bye bye love
2,3,#BITCH,BITCH


In [82]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

def filter_text(text):
    filtered=[]
    for w in text:
        if w.lower() not in stopwords.words('english'):
            filtered.append(w)
    return filtered
def lower(array):
    arr=[]
    for text in array:
        arr.append(text.lower())
    return arr
def get_pos(word):
    tag=nltk.pos_tag([word])[0][1][0]
    if tag =='J':
        return wordnet.ADJ
    elif tag =='V':
        return wordnet.VERB
    elif tag =='N':
        return wordnet.NOUN
    elif tag =='R':
        return wordnet.ADV
    else:
        return wordnet.NOUN

lem=WordNetLemmatizer()

ps=LancasterStemmer()


df['tokenized_text'] = df.apply(lambda row :lower(word_tokenize(row['clean_text'])), axis=1)


df['filteredsent'] = df['tokenized_text'].apply(lambda row : filter_text(row))

# We can either use stemmed version or lemmatized version for further analysis.
#here we used lemmatized
df['Lemmatized']=df.apply(lambda row :[lem.lemmatize(i,pos=get_pos(i)) for i in row['filteredsent']],axis=1)
# df['stemwords'] = df.apply(lambda row : [ps.stem(i) for i in row['filteredsent']],axis=1)
df

,User,Comment,clean_text,tokenized_text,filteredsent,Lemmatized
0,1,Hello bitch,Hello bitch,"[hello, bitch]","[hello, bitch]","[hello, bitch]"
1,2,bye bye love,bye bye love,"[bye, bye, love]","[bye, bye, love]","[bye, bye, love]"
2,3,#BITCH,BITCH,[bitch],[bitch],[bitch]


In [83]:
profanity_words=["bitch","fucking","asshole"]
profanity_words=lower(profanity_words)
print(profanity_words)
#use profanity_words_lem or profanity_words_stem depending on what was selected in preprocessing step
profanity_words_lem =[lem.lemmatize(i,pos=get_pos(i)) for i in profanity_words]
# profanity_words_stem =[ps.stem(i) for i in profanity_words ]
def IndexofProfanity(df,percent=False):
    feature=0
    for i in profanity_words:
        if i in df:
            feature+=1
    if percent==True:
        return feature/len(df)
    else:
        return feature
def Range(num):
    if num > 0.8:
        return "extreme"
    elif num >0.4:
        return "High"
    elif num>0.1:
        return "Low"
    elif num == 0:
        return "Clean"
    else:
        return "very Low"
       
df['Profanity_words']=df['Lemmatized'].apply(lambda row: IndexofProfanity(row))
df['Profanity_percentage']=df['Lemmatized'].apply(lambda row: IndexofProfanity(row,percent=True))
df['Profanity_level']=df['Lemmatized'].apply(lambda row: Range(IndexofProfanity(row)))


['bitch', 'fucking', 'asshole']


In [84]:
df=df.drop(columns=["clean_text","tokenized_text","filteredsent","Lemmatized"])
df.head()

,User,Comment,Profanity_words,Profanity_percentage,Profanity_level
0,1,Hello bitch,1,0.5,extreme
1,2,bye bye love,0,0.0,Clean
2,3,#BITCH,1,1.0,extreme
